**Short answers**

* **Do I need to extract features?**
  Not required. A good **TF‑IDF** (or BM25) on the **raw concatenated text** works; **adding structured attrs** (color, fabric, brand) as tokens **helps** (esp. for exact filters + cold start).

* **Better approach for “similar products”?**
  Use **sentence embeddings** (multilingual) for true semantic similarity; optionally **hybrid** = embeddings (or TF‑IDF) for recall → **rerank** with catalog constraints (same category/price) and/or **co‑purchase**.

---

### Minimal pipelines

**A) TF‑IDF (fast, strong baseline)**

```python
# text field: title + desc + attrs (weighted by duplication or prefixes)
def build_text(row):
    return f"title: {row.title} brand: {row.brand} material: {row.material} desc: {row.desc}"

corpus = df.apply(build_text, axis=1)

from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(analyzer='word',
                      ngram_range=(1,2),
                      min_df=5, max_df=0.6,
                      stop_words='swedish')  # or None + char_wb 3–5 for typos
X = vec.fit_transform(corpus)                   # items×vocab (L2 normalized)

# query vector & nearest neighbors
qv = vec.transform([build_text(query_item_row)])
scores = (X @ qv.T).A.ravel()
idx = scores.argsort()[::-1]
candidates = df.iloc[idx].head(50)             # filter: same category, price window, etc.
```

**B) Sentence embeddings (better semantics, multilingual)**

```python
# pip install sentence-transformers
from sentence_transformers import SentenceTransformer
enc = SentenceTransformer("multilingual-MiniLM")  # pick a small multilingual model
E = enc.encode(corpus, normalize_embeddings=True) # items×d

# cosine similarity
import numpy as np
q = enc.encode([build_text(query_item_row)], normalize_embeddings=True)[0]
scores = (E @ q)                                  # dot = cosine
idx = np.argsort(-scores)
candidates = df.iloc[idx].head(50)
```

**Hybrid idea:** `score = 0.7 * cosine_embed + 0.3 * cosine_tfidf`, then apply **category filter** and business rules.

---

### What you might be missing

* **Language-specific normalization** (Swedish): lowercasing, Unicode normalization, optional **stemming/lemmatization** (Snowball “swedish”), keep numerics/units (“100% bomull”, “40°”).
* **Synonym/normalization table** (e.g., “bomull”→“cotton”, “pyjamas/pajamas”).
* **Field weighting**: upweight title/brand (duplicate tokens or separate vectorizers and sum).
* **ANN index** (FAISS/Annoy) for fast retrieval at scale.
* **Blending with behavior** (co‑purchase) if you also want complements.

---

### Offline metrics (use what you have)

* **Recall\@K / HitRate\@K / NDCG\@K** using proxy positives: same **category/brand**, or **co-view/co-purchase substitutes**.
* **Category purity** of k-NN (percentage of neighbors in same category).
* **Coverage** (items with ≥1 neighbor), **diversity** (brand/category spread).
* If you have clicks: **MRR/NDCG\@K** against clicked‑next similar items.

**Tip:** even if you don’t extract columns, **include known attrs into the text** (e.g., `color=blå material=bomull`) so both TF‑IDF and embeddings “see” them.


Color/category filters at retrieval time; boost rather than hard‑filter if recall drops.
Hybrid with CF: final score = 0.7*embed_cos + 0.3*CF_sim when an anchor group (from cart/ PDP) is available.
Offline eval: category/brand/color purity@K, Recall/HitRate/NDCG@K vs. proxy “similar” pairs (co‑view/co‑purchase in same category), coverage and diversity.


# Build in-domain positives from your data (example: same-category co-purchase pairs)
pos_pairs = [(gid_a, gid_b) for (gid_a,gid_b) in co_purchase_pairs if same_category(gid_a,gid_b)]
gid2text = dict(zip(group_df.groupId, group_df.text))

from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

train_ex = [InputExample(texts=[gid2text[a], gid2text[b]]) for a,b in pos_pairs]
loader   = DataLoader(train_ex, batch_size=64, shuffle=True)

model = SentenceTransformer("KBLab/sentence-bert-swedish-cased", device="cpu")
loss  = losses.MultipleNegativesRankingLoss(model)   # contrastive retrieval loss

model.fit(train_objectives=[(loader, loss)], epochs=1)  # 1–3 epochs is often enough

# Encode & search as before; keep your soft rerank (name/brand/color boosts)
E = model.encode(group_df["text"].tolist(), normalize_embeddings=True, batch_size=64).astype("float32")


## Assemble a DS

In [25]:
import pandas as pd

art = pd.read_csv(
    "../../data/processed/articles_clean.csv",
    usecols=["sku","groupId", "name", "brand", "description", "audience", "category", "priceSEK", "color", "fabrics"],
    dtype=str
)

In [26]:
# Remove articles with priceSEK below 1 from the DataFrame and convert to numeric
art['priceSEK'] = pd.to_numeric(art['priceSEK'], errors='coerce')
art = art[art['priceSEK'] >= 1]

In [27]:
# Create price buckets based on the distribution of price_sek, using 6 buckets
price_bins = [0, 100, 300, 600, 1000, 2000, float('inf')]
price_labels = [
    'Budget',        # 0-100
    'Value',         # 100-300
    'Popular',       # 300-600
    'Premium',       # 600-1000
    'Luxury',        # 1000-2000
    'Exclusive'      # 2000+
]
art['priceband'] = pd.cut(art['priceSEK'], bins=price_bins, labels=price_labels, include_lowest=True)
art['priceband'] = art['priceband'].astype(object)
art.loc[art['priceSEK'].isna(), 'priceband'] = 'unknown'

In [28]:
with pd.option_context('display.max_columns', None, 'display.width', 0):
    display(art.sort_values("groupId"))

,sku,groupId,name,description,brand,color,audience,category,priceSEK,fabrics,priceband
11331,055522,055522,Tröja,Sticka en färgglad och trendig tröja i garnet Vera från House of Yarn!\n \nRekommenderade stickor: Stor och liten rundsticka och strumpsticka 6 och 8 mm,Gjestal Garn,NaN,dam,Tröjor,29.0,NaN,Budget
11330,055573,055573,Luva,"Sticka en trendig huva i garnet Halaus från Novita!\nRekommenderade stickor: Rundsticka 40 och 80 cm, 10 mm",Novita,vit,dam,"Mössor & hattar,Mönster",29.0,NaN,Budget
11329,055575,055575,Vantar,Sticka ett par vantar med blomstermotiv i garnet Halaus från Novita!\nRekommenderade stickor: Strumpstickor 7mm,Novita,vit,dam,Vantar,29.0,NaN,Budget
11328,055576,055576,Benvärmare,Sticka ett par trendiga benvärmare i garnet Halaus från Novita!\nRekommenderade stickor: Strumpstickor 8 och 10 mm,Novita,vit,dam,Sockor & strumpor,29.0,NaN,Budget
11315,095318,095302,Garn Drops Nepal,"Garn Drops Nepal är ett underbart tjockt luxuöst garn spunnet av 35% superfin alpaca och 65 % peruvian highland ull. Garnfibrerna är obehandlade, vilket innebär att de enbart har tvättats och inte utsatts för kemisk behandling innan färgning. Detta lyfter fram garnet naturliga egenskaper på bästa sätt och ger bättre form och textur.Blandningen lyfter fram alpacans sidenmjuka yta, samtidigt som ullen bidrar till bättre form och stabilitet. Garnet är spunnet av 3 trådar vilket ger en spännande, rustik och vacker maskbild. Garn Drops Nepal är en lättstickad/virkad kvalitet som också passar utmärkt att tova. Kvalité: 65% ull och 35% alpaca.Vikt/längd: 50 gram &#61; ca 75 m.Rekommenderade stickor: 5 mm.Stickfasthet: 10 x 10 cm &#61; 17 m x 22 v.Tvättråd: Handtvätt, max 30°.",Drops Design,svart,unknown,unknown,33.0,"['alpaca', 'ull']",Budget
...,...,...,...,...,...,...,...,...,...,...,...
14,AH2031-4547,AH2031,Stödstrumpa Herr,"FUNQ WEAR Komfortsockor MILD 11-14 mmHg Organic Cotton är graderade kompressionssockor av högsta kvalitet. Dessa är utprovade på både kvinnor och män. Det extra stödet kring ankeln ökar blodcirkulationen och motverkar svullnad i fötterna. FUNQ WEAR Komfortsockor MILD Organic Cotton uppskattas för sin färgglada design av bla resande, stillasittande, gravida samt för dig som står och går mycket på jobbet. De är lämpliga för vardagligt bruk och perfekta för varmare dagar eller för dig som föredrar ankelsockor framför knästrumpor. Komfortsockorna stickas av ekologisk bomull av högsta kvalitet. Kan krympa något efter tvätt. 55% EKO bomull, 30% polyamid och 15% lycra. Tvätt 60°. Färg grå.",Funq Wear,svart-grå,generic,"Stödstrumpor,Stödartiklar",149.0,"['bomull', 'elastan']",Value
19,AH2031-4244,AH2031,Stödstrumpa Herr,"FUNQ WEAR Komfortsockor MILD 11-14 mmHg Organic Cotton är graderade kompressionssockor av högsta kvalitet. Dessa är utprovade på både kvinnor och män. Det extra stödet kring ankeln ökar blodcirkulationen och motverkar svullnad i fötterna. FUNQ WEAR Komfortsockor MILD Organic Cotton uppskattas för sin färgglada design av bla resande, stillasittande, gravida samt för dig som står och går mycket på jobbet. De är lämpliga för vardagligt bruk och perfekta för varmare dagar eller för dig som föredrar ankelsockor framför knästrumpor. Komfortsockorna stickas av ekologisk bomull av högsta kvalitet. Kan krympa något efter tvätt. 55% EKO bomull, 30% polyamid och 15% lycra. Tvätt 60°. Färg grå.",Funq Wear,svart-grå,generic,"Stödstrumpor,Stödartiklar",149.0,"['bomull', 'elastan']",Value
20,AH3021-4244,AH3021,Stödstrumpa Herr,"FUNQ WEAR Komfortsockor MILD 11-14 mmHg Organic Cotton är graderade kompressionssockor av högsta kvalitet. Dessa är utprovade på både kvinnor och män. Det extra stödet kring ankeln ökar blodcirkulationen och motverkar svullnad i fötterna. FUNQ WEAR Komfortsockor MILD Organic Cotton uppskattas för sin färgglada design av bla resande, stillasittande, gravida samt för dig som står och går mycket på jobbet. De är lämpliga för vardagligt bruk och perfekta för varmare dagar eller för dig som föredrar an

In [29]:
# Deduplicate so that for each groupId, keep the first row for all columns except 'color', 
# which should be a list of all colors from merged rows (excluding missing/unknown/nan/none).
def merge_colors(series):
    # Remove missing/unknown/nan/none and deduplicate
    colors = [str(c).strip() for c in series if pd.notna(c) and str(c).strip().lower() not in {"", "unknown", "nan", "none"}]
    return list(sorted(set(colors))) if colors else []

art = art.sort_values("sku")  # Ensure deterministic "first" row
art = art.groupby("groupId", as_index=False).agg(
    {col: (merge_colors if col == "color" else "first") for col in art.columns if col != "sku"}
)


In [30]:
art.sample(10)

,groupId,name,description,brand,color,audience,category,priceSEK,fabrics,priceband
1033,310388,Pussel Bryggeri 1000 bitar,Pussla detta livfulla och galna spektakel där munkarna fått fnatt och övertagit bryggeriet. Nunnorna kikar försiktigt fram och konstaterar att det bryggs mycket mer vin och dryck av andra slag än vad nattvarden behagar...Högkvalitativt pussel från Piatnik med underhållande motiv. På varje pusselbit upptäcker du nya figurer och karaktärer. Tillhör en rad av humorpussel som är tecknade av illustratören Francois Ruyer. En spännande och livfull utmaning för varje pusselälskare!Material: PappAntal bitar: 1000 stStorlek färdigt: 68x48 cm,Piatnik,[],generic,"Hobbyhörnan,Pussel,Pussel 1000 bitar",239.0,None,Value
1460,550728,Dekoration Tussar med häst,"En lekfull och detaljrik juldekoration designad av Camilla Ståhl. Här sitter den glada nissen Tusse på en stolt liten häst, redo för julens äventyr. Med värme fångar figuren den magiska stämningen vi förknippar med julen – en perfekt dekoration för både stora och små som älskar traditioner med en glimt i ögat.Storlek: höjd 13cm Material: PolyresinAtt teckna, skapa och att sy har gått som en röd tråd genom hennes liv. 1992 startade Camilla eget företag, då med inriktning på sömnad. Vid sidan av sömnaden började även små vättar och figurer i lera att ta form. Genom åren har Camilla haft olika uppdrag såsom bokillustrationer, mönster till presentpapper samt formgivning av figurer och dekorationer för årets alla högtider.",Nääsgränsgården,[brun],unknown,Juldekoration,329.0,None,Popular
1267,505254,Påslakanset Hotellsatin med vävda ränder,"Underbart lyxig mjuk satin med vävda ränder som går ton-i-ton. Underbart mjukt ock skönt att vakna i varje dag.Storlek: 50x60cm , 150x210cmMaterial: 100% bomullsatin,trådtäthet 200Tvättråd: 60°",Redlunds,"[blå, brun, champagne, grön, linne, ljusgrå, marin, mörkgrå, mörkgrön, rosa, röd, sand, svart, vit]",hemmet,"Bädd (linea),Påslakanset",449.0,['satin'],Popular
1463,552001,Gardinkappa Mysan,Underbar enfärgad gardinkappa med vacker spets med hålbrodyr i nederkanten.Kombinera gärna gardinkappan med bordsduken och kuddfodralen som finns i samma serie.Storlek: 50x250 cmMaterial: 100% bomullTvätt: 30°,Linea,"[linne, röd, vit]",hemmet,Kanalkappa,249.0,None,Value
1186,432041,Pussel Julutflykt 1000 bitar,Njut av julstämningen med vårt charmiga pussel som fångar en julutflykt med hundar. Denna bild ger dig en känsla av gemenskap och glädje när människors bästa vänner följer med på en äventyrlig vinterutflykt. Perfekt för att sprida julglädje och skapa fina minnen.Högkvalitativt pappussel1000 bitar. Mått 29x21 cm.,Cobble Hill,[],generic,"Hobbyhörnan,Pussel,Pussel 1000 bitar",269.0,None,Value
1139,400590,Pussel Fuji 1000 bitar,Högkvalitativt pappussel. Storlek 42x29 cm.,Cheatwell,[],generic,"Hobbyhörnan,Pussel,Pussel 1000 bitar",149.0,None,Value
610,262519,Knäskydd med magneter,None,unknown,[vit-beige],generic,Stödartiklar,149.0,None,Value
687,270078,Bikini bh Grace,Bikini-bh med bygel och med softkupor. Reglerbara axelband. 80% polyamid 20% elastan. Handtvätt.,Damella,"[ljusblå, marin, svart, turkos]",dam,"Bh utan kupstorlek,Bikini,Badkläder,Dam",499.0,['elastan'],Popular
635,265310,Trosa,"2-pack. Vackert mönstrad ton i ton. Bra passform. Ger lite stöd för mage. 68% bomull, 25% polyester, 7% elastan. Tvätt 40º. Färg vit.",Louise,[vit],dam,"Underkläder,Trosor",208.0,"['bomull', 'polyester', 'elastan']",Value
830,280058,Vinterkänga utfällbar brodd,En tidlös klassisk vinterkänga med utfällbar brodd. Ovandel i skinn. Räfflad gummisula. Teddyfoder som håller dig varm. Dragkedja på skaftets båda sidor. Skafthöjd ca 13 cm i stl 37.,Åshild,[svart],dam,"Skor,Skor & tofflor",1500.0,['skinn'],Luxury


## 1. Build a clean text field for vectorization

In [31]:
import pandas as pd, numpy as np, unicodedata, re, ast

MISSING = {"", "unknown", "nan", "none", None}

# --- tiny helpers ---

def canon(s: str) -> str:
    """Normalizes text by converting Unicode, standardizing dashes and spaces, and trimming whitespace."""
    s = unicodedata.normalize("NFKC", str(s))
    s = re.sub(r"\u00A0", " ", s)                       # nbsp
    s = re.sub(r"[\u2010-\u2015\u2212\-]+", "-", s)     # dashes
    s = re.sub(r"\s+", " ", s).strip()
    return s

def as_list(x):
    """Convert various input types to a list of strings, handling lists, tuples, arrays, string representations of lists, and single values while filtering out missing/unknown values."""
    if isinstance(x, (list, tuple, np.ndarray, pd.Series)): return [str(v) for v in x if str(v).strip() not in MISSING]
    if str(x).strip() in MISSING: return []
    s = str(x).strip()
    if s.startswith("[") and s.endswith("]"):
        try: return [str(v) for v in ast.literal_eval(s)]
        except Exception: return [s]
    return [s]

# Use the colors as they appear in the color column, splitting on common delimiters and lowercasing
def norm_colors(x):
    toks = []
    for c in as_list(x):
        c = c.lower().strip()
        # Split on common delimiters: comma, slash, whitespace, dash
        parts = re.split(r"[/,\s]+|-", c)
        for p in parts:
            p = p.strip()
            if p and p not in MISSING:
                toks.append(p)
    # dedup preserve order
    seen, out = set(), []
    for t in toks:
        if t and t not in seen:
            seen.add(t); out.append(t)
    return out

# materials,  as_list and dedup (preserve order)
def norm_materials(x):
    seen, out = set(), []
    for m in as_list(x):
        t = m.strip()
        if t and t not in seen:
            seen.add(t); out.append(t)
    return out

def norm_categories(x):
    cats = [canon(c) for c in str(x).split(",") if str(c).strip() not in MISSING]
    # dedup preserve order
    seen, out = set(), []
    for c in cats:
        if c and c.lower() not in seen:
            seen.add(c.lower()); out.append(c)
    return out

# --- build text per groupId (natural sentences for embeddings) ---
art = art.copy()
art["colors_norm"]    = art["color"].apply(norm_colors)
art["materials_norm"] = art["fabrics"].apply(norm_materials)
art["categories_norm"]= art["category"].apply(norm_categories)

def build_text_embed(r):
    """Extract and normalize product attributes (name, description, brand, audience, priceband, price, categories, colors, materials) from a product record for text embedding generation."""
    name = canon(r.get("name",""))
    desc = canon(r.get("description",""))
    brand = canon(r.get("brand","")) if r.get("brand","") not in MISSING else ""
    audience = r.get("audience","")
    audience = audience if str(audience).lower() not in MISSING and audience!="generic" else ""
    priceband = r.get("priceband","")
    priceband = priceband if str(priceband).lower() not in MISSING else ""
    price = r.get("priceSEK", "")
    cats = r.get("categories_norm", [])
    cols = r.get("colors_norm", [])
    mats = r.get("materials_norm", [])

    # combining product name/description with metadata (brand, category, audience, materials, colors, price, priceband) in Swedish labels for text embedding generation.
    parts = []
    if name: parts.append(f"{name}.")
    if desc: parts.append(desc)
    meta = []
    if brand:    meta.append(f"Varumärke: {brand}.")
    if cats:     meta.append("Kategori: " + ", ".join(cats) + ".")
    if audience: meta.append(f"Målgrupp: {audience}.")
    if mats:     meta.append("Material: " + ", ".join(mats) + ".")
    if cols:     meta.append("Färger: " + ", ".join(cols) + ".")
    if pd.notna(price) and str(price).strip() not in MISSING:
        meta.append(f"Pris: {int(round(float(price)))} SEK.")
    if priceband: meta.append(f"Prisnivå: {priceband}.")
    return " ".join(parts + meta).strip()

# creates a text embedding, combining product attributes into a single text string for similarity search.
art["text"] = art.apply(build_text_embed, axis=1)

# Include all products regardless of text description and select relevant columns for similarity search
group_df = art[
    ["groupId","text","colors_norm","materials_norm","categories_norm","brand","priceband"]
].reset_index(drop=True)

# Extract the text column from group_df as a list to create a corpus for TF-IDF vectorization
corpus = group_df["text"].tolist()

In [34]:
pd.set_option('display.max_colwidth', None)
group_df.sample(10)

,groupId,text,colors_norm,materials_norm,categories_norm,brand,priceband
213,260232,"Stödstrumpbyxor. Medicinska strumpbyxor från Louise. Otrolig elasticitet som formar och ger stödeffekt åt stuss och ben. 60 den med extra tå- och hälförstärkning. Extra kraftig byxdel med bred kil bak. 80% polyamid, 20% pur. Fintvätt 40. Färg solbrun. Varumärke: Louise. Kategori: Stödstrumpor. Målgrupp: dam. Material: None. Färger: beige. Pris: 159 SEK. Prisnivå: Value.",[beige],[None],[Stödstrumpor],Louise,Value
1564,888880,extra. None Material: None. Pris: 150 SEK. Prisnivå: Value.,[],[None],[],unknown,Value
1330,530000,Fraktavgift företags. None Material: None. Pris: 99 SEK. Prisnivå: Budget.,[],[None],[],unknown,Budget
997,293472,"Säng ryggstöd. Flyttbart sittstöd till sängen. Målad metallstomme med textil klädsel med en mindre kudde. Lutningen går att vinkla 5 steg. Mått 63x9x61 cm. Vikt 2,5 kg. Varumärke: Caremax. Kategori: Bädd, Stödartiklar. Målgrupp: hemmet. Material: None. Pris: 498 SEK. Prisnivå: Popular.",[],[None],"[Bädd, Stödartiklar]",Caremax,Popular
617,263285,"Bh utan bygel Alice. Formpressad mjukbehå från Trofé utan bygel med elegant präglat mönster. Bred skön resår under byst för bästa stöd och komfort. Reglerbara, stadiga axelband. 94% polyamid 6% elastan. Tvätt 40. Färg vit. Varumärke: Trofé. Kategori: Bh utan bygel, Bh, Underkläder. Målgrupp: dam. Material: elastan. Färger: svart, vit. Pris: 249 SEK. Prisnivå: Value.","[svart, vit]",[elastan],"[Bh utan bygel, Bh, Underkläder]",Trofé,Value
1090,342699,"Pussel Premium Plus Lauterbrunnen 1000 Bitar. Utforska den natursköna skönheten i Lauterbrunnen med detta fantastiska pussel från Trefl! Detta 1000-bitars Premium Plus-pussel fångar den idylliska dalen med sina majestätiska vattenfall och charmiga alpbyar, omgiven av de imponerande schweiziska Alperna. Varje bit är noggrant utformad för att ge en utmanande och tillfredsställande pusselupplevelse. Perfekt för avkoppling och en inspirerande dekoration när det är färdiglagt. Försvinn in i denna vackra landskapsbild medan du pusslar. Högkvallitativt pappussel.1000 bitar Varumärke: Trefl. Kategori: Hobbyhörnan, Pussel, Pussel 1000 bitar. Material: None. Färger: beige. Pris: 269 SEK. Prisnivå: Value.",[beige],[None],"[Hobbyhörnan, Pussel, Pussel 1000 bitar]",Trefl,Value
703,270197,"Bikinitrosa Brigitte. Klassisk Taibyxa med lite lägre midja. Damellaloggan finns på en liten skylt längst ner till vänster. 80% polyamid, 20% elastan. Handtvätt. Färg svart Varumärke: Damella. Kategori: Bikini, Dam. Målgrupp: dam. Material: elastan. Färger: svart, grå. Pris: 329 SEK. Prisnivå: Popular.","[svart, grå]",[elastan],"[Bikini, Dam]",Damella,Popular
1567,970101,Frakt & exp.avgift. None Material: None. Pris: 55 SEK. Prisnivå: Budget.,[],[None],[],unknown,Budget
819,280012,"Sko Rory skinn. En snygg och klassisk sko i skinn som erbjuder total komfort för alla typer av fötter! Inga sömmar i tådelen och en polstrad kant runt öppningen. Rory finns i 2 bredder per storlek vilket normalt inte går att hitta i vanliga skobutiker. Dekorativa stickningar och kardborreslejf. Stor öppning gör skon enkel att komma i. Uttagbar innesula ger plats åt egna inlägg. Färg svart.Denna artikel är för en mycket bred fot, hög i tådelen. För dig som brukar ha mycket svårt att hitta en snygg sko i rätt storlek och bredd. Sök även artikel nr 280010 som är för normal till lite bredare fot. Varumärke: Good Living. Kategori: Skor. Målgrupp: dam. Material: skinn. Färger: svart. Pris: 1598 SEK. Prisnivå: Luxury.",[svart],[skinn],[Skor],Good Living,Luxury
698,270119,"Bikinitrosa Milla. Bikinitrosa från Damella. Trosan är av triangeldesign med snörning i sidorna. Damella loggan finns på diskret skylt på vänster höft framtill. 80% polyamid 20% elastan. Handtvätt. Färg svart. Varumärke: Damella. Kategori: Bikini, Badkläder. Målgrupp: dam. Material: elastan. Färger: blå, svart. Pris: 329 SEK. Prisnivå: Popular.","[blå, svart]",[elastan],"[Bikini, Badkläder]",Damella,Popular


In [35]:
# === TF‑IDF (word bigrams + char 3–5 for robustness) + cosine KNN ===
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize, MultiLabelBinarizer
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import hstack, csr_matrix
import numpy as np
import pandas as pd
import re

# --- build text features (same as you had) ---
group_df = group_df.reset_index(drop=True)
corpus   = group_df["text"].astype(str).tolist()
gids     = group_df["groupId"].to_numpy()
colors   = group_df["colors_norm"].apply(lambda x: set(x or [])).to_numpy()
cats_col = group_df["categories_norm"].apply(lambda x: x or []).tolist()

vec_w = TfidfVectorizer(analyzer="word", ngram_range=(1,2), min_df=2, max_df=0.8)
vec_c = TfidfVectorizer(analyzer="char_wb", ngram_range=(3,5), min_df=3)
Xw = vec_w.fit_transform(corpus)
Xc = vec_c.fit_transform(corpus)

# --- category block as multi-hot (sparse) ---
mlb = MultiLabelBinarizer(sparse_output=True)
C = mlb.fit_transform(cats_col)           # shape: [n_docs, n_cats]
C = csr_matrix(C)                         # ensure CSR for ops

# Row-normalize each block so weights are comparable, then apply block weights
Xw_n = normalize(Xw)        # word TF-IDF
Xc_n = normalize(Xc)        # char TF-IDF
C_n  = normalize(C)         # categories (binary)

W_WORD, W_CHAR, W_CAT = 1.0, 1.0, 3.0     # <= raise W_CAT to prioritize categories more
X = normalize(hstack([W_WORD*Xw_n, W_CHAR*Xc_n, W_CAT*C_n]).tocsr())

# KNN over the fused space
POOL = min(201, len(group_df))
nn = NearestNeighbors(metric="cosine", algorithm="brute", n_neighbors=POOL).fit(X)


# --- search helpers ---
def search_by_gid(gid, k=50, require_color=None):
    i = int(np.where(gids == gid)[0][0])
    d, idx = nn.kneighbors(X[i], n_neighbors=min(k+1, len(gids)))  # +1 for self
    idx, d = idx[0], d[0]
    m = idx != i
    idx, d = idx[m], d[m]
    if require_color:
        m2 = np.array([require_color in colors[j] for j in idx], bool)
        idx, d = idx[m2], d[m2]
    sim = 1.0 - d
    return group_df.iloc[idx].assign(similarity=sim).head(k)

def search_by_text(q, k=50, require_color=None):
    qv = normalize(hstack([vec_w.transform([q]), vec_c.transform([q])]))
    d, idx = nn.kneighbors(qv, n_neighbors=min(k, len(gids)))
    idx, d = idx[0], d[0]
    if require_color:
        m2 = np.array([require_color in colors[j] for j in idx], bool)
        idx, d = idx[m2], d[m2]
    sim = 1.0 - d
    return group_df.iloc[idx].assign(similarity=sim).head(k)


In [36]:
# Build neighbors for all items (self + 10)
POOL = min(11, len(group_df))
dists, idxs = nn.kneighbors(X, n_neighbors=POOL)

gids = group_df["groupId"].to_numpy()
src  = np.repeat(gids, POOL)
rec  = gids[idxs.ravel()]
sim  = 1.0 - dists.ravel()  # cosine similarity

recs = pd.DataFrame({"src_groupId": src, "rec_groupId": rec, "similarity": sim})
recs = recs[recs["src_groupId"] != recs["rec_groupId"]].copy()

# keep top-10 per source
recs["rec_rank"] = (
    recs.groupby("src_groupId")["similarity"].rank(method="first", ascending=False).astype(int)
)
recs = recs[recs["rec_rank"] <= 10].sort_values(["src_groupId","rec_rank"])

# similarity cutoff — calibrate by quantile (e.g., keep top 80% most similar)
SIM_CUT = recs["similarity"].quantile(0.01)  # keep top 99,9% (adjust as needed)
recs = recs[recs["similarity"] >= SIM_CUT].copy()

# attach metadata for review
meta_cols = [c for c in ["text","brand","categories_norm","colors_norm","priceband"]
             if c in group_df.columns]
meta = group_df.set_index("groupId")[meta_cols]
recs = recs.join(meta.add_prefix("src_"), on="src_groupId") \
           .join(meta.add_prefix("rec_"), on="rec_groupId")

recs.to_csv("tfidf_similar_items_top10.csv", index=False)


In [37]:
# Find src_groupId that have less than one rec (i.e., zero recs)
src_counts = recs["src_groupId"].value_counts()
no_rec_srcs = src_counts[src_counts < 3]
print("src_groupId with less than one rec:", no_rec_srcs.index.tolist())


src_groupId with less than one rec: ['559097', '360140', '305043', '549005', '549183']


In [38]:
recs[recs["src_groupId"] == "260406"]

,src_groupId,rec_groupId,similarity,rec_rank,src_text,src_brand,src_categories_norm,src_colors_norm,src_priceband,rec_text,rec_brand,rec_categories_norm,rec_colors_norm,rec_priceband
2707,260406,264804,0.817893,1,"Framknäppt mjuk bh. Framknäppt komfort-bh med microfiber, ett verkligt skönt och funktionsdugligt plagg. Brett ryggband ger stöd åt ryggen. Breda axelband. Mjuka kantband. Den elastiska kupan passar både till B, C och D-kupa. Fintvätt 30.87% polyester och 13% elastan. Varumärke: Louise. Kategori: Bh utan bygel, Bh-toppar, Framknäppt bh, Bh, Bh utan kupstorlek, Underkläder. Målgrupp: dam. Material: mikrofiber, polyester, elastan. Färger: beige, svart, vit. Pris: 198 SEK. Prisnivå: Value.",Louise,"[Bh utan bygel, Bh-toppar, Framknäppt bh, Bh, Bh utan kupstorlek, Underkläder]","[beige, svart, vit]",Value,"Framknäppt mjuk bh i bomull. B &#61; BCD, E &#61; EF, G&#61;GH Skön framknäppt bomulls-bh från Glamoris med rygg i genombruten spets som andas. Släta kupor med stretch, anpassar formen. T-rygg håller de mjuka, sömlösa axelbanden på plats. Utformad för komfort - både för dagen och på natten. 60% bomull, 30% polyamid, 10% elastan. Fintvätt 40°. Varumärke: Glamorise. Kategori: Bh utan bygel, Bh-toppar, Framknäppt bh, Bh, Underkläder. Målgrupp: dam. Material: bomull, elastan. Färger: beige, rosa, vit. Pris: 439 SEK. Prisnivå: Popular.",Glamorise,"[Bh utan bygel, Bh-toppar, Framknäppt bh, Bh, Underkläder]","[beige, rosa, vit]",Popular
2708,260406,261324,0.800178,2,"Framknäppt mjuk bh. Framknäppt komfort-bh med microfiber, ett verkligt skönt och funktionsdugligt plagg. Brett ryggband ger stöd åt ryggen. Breda axelband. Mjuka kantband. Den elastiska kupan passar både till B, C och D-kupa. Fintvätt 30.87% polyester och 13% elastan. Varumärke: Louise. Kategori: Bh utan bygel, Bh-toppar, Framknäppt bh, Bh, Bh utan kupstorlek, Underkläder. Målgrupp: dam. Material: mikrofiber, polyester, elastan. Färger: beige, svart, vit. Pris: 198 SEK. Prisnivå: Value.",Louise,"[Bh utan bygel, Bh-toppar, Framknäppt bh, Bh, Bh utan kupstorlek, Underkläder]","[beige, svart, vit]",Value,"Bh topp Heidi. Heidi en mjuk och skön bh-topp i trikåmaterial frånTrofé. Bh:n är utan sömmar och knäppning. Bh:n dras över huvudet och har inga skavande detaljer eller byglar vilket också gör den utmärkt att sova i. Kuporna är vadderade men vadderingen går att plocka bort. Bh:n har brett resår nertill och breda axelband som ger extra komfort. Rynkeffekt mellan brösten. 90% nylon, 10% elastan. Handtvätt. Varumärke: Trofé. Kategori: Bh utan bygel, Bh-toppar, Bh, Bh utan kupstorlek, Underkläder. Målgrupp: dam. Material: nylon, elastan. Färger: svart, vit. Pris: 179 SEK. Prisnivå: Value.",Trofé,"[Bh utan bygel, Bh-toppar, Bh, Bh utan kupstorlek, Underkläder]","[svart, vit]",Value
2709,260406,260423,0.756612,3,"Framknäppt mjuk bh. Framknäppt komfort-bh med microfiber, ett verkligt skönt och funktionsdugligt plagg. Brett ryggband ger stöd åt ryggen. Breda axelband. Mjuka kantband. Den elastiska kupan passar både till B, C och D-kupa. Fintvätt 30.87% polyester och 13% elastan. Varumärke: Louise. Kategori: Bh utan bygel, Bh-toppar, Framknäppt bh, Bh, Bh utan kupstorlek, Underkläder. Målgrupp: dam. Material: mikrofiber, polyester, elastan. Färger: beige, svart, vit. Pris: 198 SEK. Prisnivå: Value.",Louise,"[Bh utan bygel, Bh-toppar, Framknäppt bh, Bh, Bh utan kupstorlek, Underkläder]","[beige, svart, vit]",Value,"Framknäppt bh. Formsydd framknäppt bh som sitter fantastisk skönt och bekvämt. Brett ryggparti som ger lite stöd och slätar ut. Reglerbara och mjuka axelband. 90% polyamid, 10% elastan. Fintvätt 30°. Varumärke: Louise. Kategori: Bh utan bygel, Framknäppt bh, Bh, Underkläder. Målgrupp: dam. Material: elastan. Färger: vit. Pris: 229 SEK. Prisnivå: Value.",Louise,"[Bh utan bygel, Framknäppt bh, Bh, Underkläder]",[vit],Value
2710,260406,260372,0.755895,4,"Framknäppt mjuk bh. Framknäppt komfort-bh med microfiber, ett verkligt skönt och funktionsdugligt plagg. Brett rygg

## 2. Vectorize with TF-IDF (Term Frequency × Inverse Document Frequency)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    lowercase=True,
    ngram_range=(1, 2), #use unigrams + bigrams
    min_df=1,
    strip_accents=None,   # keep å/ä/ö
    # sublinear_tf=True,  # optional
    # dtype=np.float32,   # optional memory saver
    token_pattern=r'(?u)\b\w+\b'
)
X_tfidf = tfidf.fit_transform(group_df["text"])
X_tfidf.shape

(1573, 46734)

## 3. Singular Value Decomposition + L2 normalize

TF-IDF gives precise but sparse signals; SVD compresses & generalizes them.
L2-norm makes nearest-neighbor search stable and comparable across items.

In [16]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize

n_components = min(128, max(2, X_tfidf.shape[1]-1)) 
svd = TruncatedSVD(n_components=n_components, random_state=0)
X_svd = svd.fit_transform(X_tfidf)
X_emb = normalize(X_svd)
X_emb.shape


(1573, 128)

## 4. Build 10-nearest neighbors (cosine) and return a small recs table


* It’s the **cosine of the angle** between vectors $a$ and $b$:

  $$
  \text{cosine\_sim}(a,b)=\frac{a\cdot b}{\|a\|\;\|b\|}
  $$

  * $=1$ → same direction (very similar)
  * $=0$ → orthogonal (unrelated)
  * $=-1$ → opposite (rare with TF-IDF since values are ≥0)

* After we **L2-normalize** vectors, cosine similarity becomes just the **dot product**.

* In scikit-learn, `metric="cosine"` actually computes **cosine distance**:

  $$
  \text{cosine\_dist} = 1 - \text{cosine\_sim}
  $$

  That’s why in the code we convert back with `similarity = 1 - d`.

Why we use it: it’s **scale-invariant** (ignores length), so two SKUs with similar wording but different text lengths still match well.


In [17]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

POOL = min(11, len(group_df))
nn = NearestNeighbors(metric="cosine", n_neighbors=POOL).fit(X_emb)
dists, idxs = nn.kneighbors(X_emb)

groupid_arr = group_df["groupId"].values

# Build all recommendations, then filter out self-recommendations
src_groupids = np.repeat(groupid_arr, POOL)
rec_groupids = groupid_arr[idxs.ravel()]
similarities = 1 - dists.ravel()

recs = pd.DataFrame({
    "src_groupId": src_groupids,
    "rec_groupId": rec_groupids,
    "similarity": similarities
})

# Remove self-recommendations
recs = recs[recs["src_groupId"] != recs["rec_groupId"]].copy()

# For each src_groupId, keep only the top POOL-1 recommendations (in case of ties or accidental duplicates)
recs["rec_rank"] = (
    recs.groupby("src_groupId")["similarity"]
    .rank(method="first", ascending=False)
    .astype(int)
)
recs = recs[recs["rec_rank"] <= POOL-1]

recs.head(10)

,src_groupId,rec_groupId,similarity,rec_rank
1,055522,055573,0.947612,1
2,055522,055576,0.944613,2
3,055522,055575,0.935065,3
4,055522,170005,0.881895,4
5,055522,294785,0.576481,5
6,055522,305043,0.555728,6
7,055522,313032,0.549852,7
8,055522,095302,0.476616,8
9,055522,294793,0.426344,9
10,055522,409090,0.426119,10


In [18]:
recs.sample(15, random_state=42)

,src_groupId,rec_groupId,similarity,rec_rank
12977,431017,393108,0.589174,8
3503,260844,260164,0.770456,5
6346,261998,265310,0.571110,10
10348,291120,291088,0.673962,8
97,141412,261495,0.460513,9
9440,290135,292037,0.808479,2
14402,525027,525035,0.995877,3
1913,260084,261690,0.437207,10
15033,538053,538071,0.782572,7
1510,241091,240012,0.940151,3


In [19]:
target = "190041"  # example groupId

# pick columns and collapse art to one row per groupId
keep = [c for c in ["groupId","name","name.1","color","audience","category","priceband","description"] if c in art.columns]
details = art.drop_duplicates("groupId")[keep].copy()

view = (
    recs.loc[recs["src_groupId"].eq(target), ["rec_groupId","rec_rank","similarity"]]
        .merge(details, left_on="rec_groupId", right_on="groupId", how="left")
        .drop(columns=["groupId"])  # joined key
        [["rec_rank","similarity","rec_groupId"] + [c for c in keep if c != "groupId"]]
        .sort_values("rec_rank")
        .reset_index(drop=True)
)

view


,rec_rank,similarity,rec_groupId,name,color,audience,category,priceband,description
0,1,0.889783,500389,Sovkudde Lyx,[vit],hemmet,"Bäddtillbehör,Bädd (linea)",Value,"Mjuk och bekväm sovkudde med ett elegant yttertyg av tätvävd polyester/bomull och fyllning av silikoniserad bollfiber av polyester, 500g. Storlek: 50x60 cmFärg: vitMaterial: yttertyg polyester/bomull, fyllning silikoniserad bollfiber av polyesterTvätt: 60°, torktumlas."
1,2,0.861629,521879,Innerkudde Rund,[vit],hemmet,"Innerkuddar,Bädd (linea)",Value,"Rund innerkudde med ett yttertyg av bomull och polyester, fyllning av polyester. Kudden är Oeko-Tex certifierad vilket är ett intyg på att den inte orsakar hälsoproblem. Storlek: ø50 cm, fyllnad 340 gFärg: vitMaterial: bomull/polyester"
2,3,0.845581,500397,Täcke Lyx,[vit],hemmet,"Täcken,Bädd (linea)",Premium,"Mjukt och lyxigt quiltstickat täcke med ett elegant yttertyg i tätvävd polyester/bomull med en fyllning av silikoniserad polyester, hålfiber 1000g eller 700g. Bandkant i yttertyg. Storlek: 150x200 cmFärg: vitMaterial: yttertyg polyester/bomull, fyllning silikoniserad polyesterTvätt: 60°, torktumlas."
3,4,0.830786,576223,Sovkudde Låg,[vit],hemmet,"Innerkuddar,Bädd (linea)",Value,"Mjuk och bekväm sovkudde med ett yttertyg av tätvävd polyester och fyllning av silikoniserad bollfiber av polyester, Innerkudden har ett bandkant runt alla fyra sidor.Fyllning: 50x60cm 400g, 50x70cm 500g, 65x90cm 700gFärg: vitMaterial: yttertyg 100% polyester, fyllning silikoniserad bollfiber av polyesterTvätt: 60°, torktumlas."
4,5,0.821790,576249,Sovkudde Hög,[vit],hemmet,"Innerkuddar,Bädd (linea),Kuddar",Value,"Mjuk och bekväm sovkudde med ett yttertyg av tätvävd polyester och fyllning av silikoniserad bollfiber av polyester. Innerkudden har ett bandkant runt alla fyra sidor.Fyllning: 50x60cm 600g, 50x70cm 700g, 65x90cm 1100gFärg: vitMaterial: yttertyg 100% polyester, fyllning silikoniserad bollfiber av polyesterTvätt: 60°, torktumlas."
5,6,0.819241,579009,Täcke Kingsize,[vit],hemmet,"Täcken,Bädd (linea)",Popular,"Mjukt och behagligt quiltat täcke med bandkant. Yttertyget är av tätvävd polyester och fyllning av silikoniserad polyester. Hålfiber 800g. Storlek: 230x220 cmFärg: vitMaterial: yttertyg 100% polyester, fyllning silikoniserad polyesterTvätt: 60°, torktumlas."
6,7,0.818103,576231,Sovkudde Medium,[vit],hemmet,"Innerkuddar,Bädd (linea),Kuddar",Value,"Mjuk och bekväm sovkudde med ett yttertyg av tätvävd polyester och fyllning av silikoniserad bollfiber av polyester. Innerkudden har ett bandkant runt alla fyra sidor.Fyllning: 50x60cm 500g, 50x70cm 600g, 65x90cm 900gFärg: vitMaterial: yttertyg 100% polyester, fyllning silikoniserad bollfiber av polyesterTvätt: 60°, torktumlas."
7,8,0.799948,588970,Sovkudde Duni,[vit],hemmet,"Innerkuddar,Bädd (linea)",Value,"Mjuk och bekväm innerkudde. Yttertyget är gjort av 100% duntät bomull och fyllningen av 95% andfjädrar och 5% anddun. Storlek: 50x60 cmFärg: vitMaterial: yttertyg 100% bomull, fyllning 95% andfjädrar, 5% anddunTvätt: 60°, torktumlas."
8,9,0.723638,500355,Madrasskydd i 6 storlekar,[vit],hemmet,"Bädd,Bäddtillbehör,Bädd (linea)",Value,"Madrasskydd i en tålig bomull- och polyesterblandning. Madrasskyddet är quiltat och formsytt samt hålls på plats med hjälp av resårband på kortsidorna. Finns i flera olika storlekar.Material: 80% Polyester, 20% Bomull. Fyllning 100% PolyesterTvätt: 60°"
9,10,0.699642,522710,Sovkudde Hotell,[vit],hemmet,"Täcken,Bäddtillbehör,Bädd (linea),Kuddar,Innerkuddar",Popular,"Mjuk och lyxig sovkudde med grå passpoal. Yttertyget är gjort av bomullscambric och fyllningen av hålfiber/microfiber, 760 g som ger den ultimata hotellkänslan. Storlek: 50x70 cmFärg: vitMaterial: yttertyg bomullscambric, fyllning hålfiber/microfiber - OEKO-TEXTvätt: 60°, tål ej blekmedel. Lätt tumling."


In [20]:
# Visualize the recommendations for the target itself (i.e., show the row for the target)
cols = ["rec_rank", "similarity", "rec_groupId"] + [c for c in keep if c != "groupId"]
if "description" not in cols:
    cols.append("description")
view_target = (
    details.loc[details["groupId"] == target]
        .assign(rec_rank=1, similarity=1.0, rec_groupId=target)
        [cols]
        .reset_index(drop=True)
)
view_target


,rec_rank,similarity,rec_groupId,name,color,audience,category,priceband,description
0,1,1.0,190041,Innerkudde,[vit],hemmet,"Kuddar,Innerkuddar,Bädd (linea)",Budget,"Fyllnadskudde till kuddfodral med ett yttertyg av polyester/bomull och fyllning av polyesterfibrer. Innerkudden finns i flera olika storlekar för att matcha alla våra olika kuddfodral. Färg: vitMaterial: yttertyg polyester/bomull, fyllning polyester"


In [22]:
out = (recs
       .sort_values(['src_groupId','rec_rank'])
       .groupby('src_groupId')['rec_groupId']
       .apply(list)
       .reset_index(name='recs'))

out.to_csv("../../data/predictions/vector_similarity_recommendations.csv", index=False)


In [24]:
out.sample(10)

,src_groupId,recs
103,210832,"[210834, 270799, 210756, 210186, 210784, 210777, 210763, 210695, 210835, 210814]"
869,290172,"[290104, 290007, 290012, 290207, 290993, 290286, 290285, 290117, 290178, 290267]"
1274,509164,"[509130, 590461, 512219, 546102, 525029, 546117, 546113, 546112, 543046, 553124]"
515,261813,"[261815, 261817, 261866, 261869, 261950, 260256, 261436, 261822, 260276, 260274]"
982,292929,"[292930, 270610, 270599, 270609, 270611, 270543, 270598, 270597, 951029, 290183]"
837,281493,"[281098, 280051, 280005, 281519, 280006, 281477, 280053, 281410, 280038, 281675]"
1324,527002,"[536763, 106065, 206524, 206532, 350600, 350923, 342526, 280032, 293886, 290255]"
390,261310,"[261690, 266627, 260819, 260839, 260069, 260425, 261472, 260822, 260095, 261718]"
116,220220,"[280032, 262527, 970100, 200071, 262519, 970101, 508311, 530000, 110156, 290267]"
926,290294,"[292086, 292078, 290261, 290016, 290039, 290304, 290229, 292839, 292706, 294389]"
